<a href="https://colab.research.google.com/github/Ximena5745/Desempeno_Institucional/blob/main/Consolidado_Ind.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import os
import glob
import re
import ast
import pandas as pd
from html import unescape
import json

In [2]:
!git clone https://github.com/Ximena5745/Desempeno_Institucional.git

fatal: destination path 'Desempeno_Institucional' already exists and is not an empty directory.


In [3]:
os.chdir("/content/Desempeno_Institucional/Data")

In [4]:
# 3) Cargar todos los Excel del directorio, excluyendo cualquier archivo de consolidado previo
archivos_excel = [f for f in (glob.glob("*.xlsx") + glob.glob("*.xls")) if "consolidado" not in f.lower()]


In [8]:
dfs = []
for archivo in archivos_excel:
    try:
        df_tmp = pd.read_excel(archivo)
        df_tmp["Archivo"] = archivo
        dfs.append(df_tmp)
    except Exception as e:
        print(f"⚠️ No se pudo leer {archivo}: {e}")

if not dfs:
    raise RuntimeError("No se encontraron archivos Excel para consolidar.")

# 4) Unir y estandarizar columnas
df_todos = pd.concat(dfs, ignore_index=True).drop_duplicates()
df_todos.columns = df_todos.columns.str.strip().str.lower()

# 5) Limpieza pedida
cols_a_eliminar = ["dependencia", "incidencia", "objectivos_relacionados"]
df_todos = df_todos.drop(columns=[c for c in cols_a_eliminar if c in df_todos.columns], errors="ignore")

if "fecha" in df_todos.columns:
    df_todos = df_todos[df_todos["fecha"].notna() & (df_todos["fecha"].astype(str).str.strip() != "")]

if "clasificacion" in df_todos.columns:
    df_todos["clasificacion"] = df_todos["clasificacion"].astype(str).map(lambda s: unescape(s))
    df_todos["clasificacion"] = df_todos["clasificacion"].replace({"Estrat&eacute;gico": "Estratégico"})


In [10]:
# -------------------------
# Funciones auxiliares
# -------------------------
def parse_variables_cell(x):
    """Convierte 'variables' de Serie Única a lista de dicts."""
    if x is None or (isinstance(x, float) and pd.isna(x)):
        return []
    if isinstance(x, list):
        return [ {"valor": i.get("valor"), "nombre": i.get("nombre"), "simbolo": i.get("simbolo")} for i in x if isinstance(i, dict) ]
    s = str(x).strip()
    if s == "" or s.lower() in {"nan", "none", "null", "[]"}:
        return []
    try:
        pyobj = ast.literal_eval(s.replace("null", "None").replace("true", "True").replace("false", "False"))
        if isinstance(pyobj, list):
            return [ {"valor": i.get("valor"), "nombre": i.get("nombre"), "simbolo": i.get("simbolo")} for i in pyobj if isinstance(i, dict) ]
    except Exception:
        pass
    bloques = re.findall(r"\{[^{}]*\}", s)
    out = []
    for b in bloques:
        m_val = re.search(r"[\'\"]?valor[\'\"]?\s*:\s*([\-+]?\d+(?:[.,]\d+)?)", b, flags=re.IGNORECASE)
        m_nom = re.search(r"[\'\"]?nombre[\'\"]?\s*:\s*[\'\"](.*?)[\'\"]", b, flags=re.IGNORECASE)
        m_sim = re.search(r"[\'\"]?simbolo[\'\"]?\s*:\s*[\'\"](.*?)[\'\"]", b, flags=re.IGNORECASE)
        valor = None
        if m_val:
            v = m_val.group(1).replace(",", ".")
            try:
                valor = float(v)
            except Exception:
                valor = v
        nombre = m_nom.group(1) if m_nom else None
        simbolo = m_sim.group(1) if m_sim else None
        if any([m_val, m_nom, m_sim]):
            out.append({"valor": valor, "nombre": nombre, "simbolo": simbolo})
    return out

def parse_multiserie_cell(x):
    """Convierte celda de 'series' a lista de dicts con meta, nombre, resultado y variables."""
    if x is None or (isinstance(x, float) and pd.isna(x)):
        return []
    if isinstance(x, list):
        return x
    try:
        return ast.literal_eval(str(x))
    except Exception:
        return []

# -------------------------
# 6) Serie Única
# -------------------------
if "tipo" in df_todos.columns and "variables" in df_todos.columns:
    mask_serie = df_todos["tipo"].astype(str).str.strip().str.lower() == "serie unica"
    if mask_serie.any():
        vars_list = df_todos.loc[mask_serie, "variables"].apply(parse_variables_cell)
        max_vars = int(vars_list.map(lambda lst: len(lst) if isinstance(lst, list) else 0).max())
        new_cols = []
        for i in range(1, max_vars + 1):
            for base in ("Variable_Valor", "Variable_Nombre", "Variable_Simbolo"):
                col = f"{base}_{i}"
                new_cols.append(col)
                if col not in df_todos.columns:
                    df_todos[col] = pd.NA
        def expand_row(lst, max_n):
            data = {}
            if not isinstance(lst, list):
                return pd.Series({c: pd.NA for c in new_cols})
            for i in range(min(max_n, len(lst))):
                item = lst[i] if isinstance(lst[i], dict) else {}
                data[f"Variable_Valor_{i+1}"]  = item.get("valor", pd.NA)
                data[f"Variable_Nombre_{i+1}"] = item.get("nombre", pd.NA)
                data[f"Variable_Simbolo_{i+1}"]= item.get("simbolo", pd.NA)
            for i in range(len(lst)+1, max_n+1):
                data.setdefault(f"Variable_Valor_{i}", pd.NA)
                data.setdefault(f"Variable_Nombre_{i}", pd.NA)
                data.setdefault(f"Variable_Simbolo_{i}", pd.NA)
            return pd.Series(data)
        expanded = vars_list.apply(lambda lst: expand_row(lst, max_vars))
        df_todos.loc[mask_serie, expanded.columns] = expanded.values

# -------------------------
# 6B) Multiserie (optimizado)
# -------------------------
if "tipo" in df_todos.columns and "series" in df_todos.columns:
    mask_multi = df_todos["tipo"].astype(str).str.strip().str.lower() == "multiserie"
    if mask_multi.any():
        series_list = df_todos.loc[mask_multi, "series"].apply(parse_multiserie_cell)

        # Máximo de series y variables por serie
        max_series = int(series_list.map(lambda lst: len(lst) if isinstance(lst, list) else 0).max())
        max_vars_per_series = {i: 0 for i in range(1, max_series+1)}
        for lst in series_list:
            if isinstance(lst, list):
                for idx, serie in enumerate(lst, start=1):
                    vars_len = len(serie.get("variables", [])) if isinstance(serie.get("variables"), list) else 0
                    if vars_len > max_vars_per_series[idx]:
                        max_vars_per_series[idx] = vars_len

        # Crear columnas necesarias
        new_cols_multi = []
        for s in range(1, max_series + 1):
            new_cols_multi += [f"Meta_{s}", f"Nombre_{s}", f"Resultado_{s}"]
            for v in range(1, max_vars_per_series[s] + 1):
                new_cols_multi += [
                    f"Var_Valor_{s}_{v}",
                    f"Var_Nombre_{s}_{v}",
                    f"Var_Simbolo_{s}_{v}"
                ]
        for col in new_cols_multi:
            if col not in df_todos.columns:
                df_todos[col] = pd.NA

        # Expansión por fila
        def expand_multiserie_row(lst):
            data = {}
            if not isinstance(lst, list):
                return pd.Series({c: pd.NA for c in new_cols_multi})
            for s_idx, serie in enumerate(lst, start=1):
                data[f"Meta_{s_idx}"] = serie.get("meta", pd.NA)
                data[f"Nombre_{s_idx}"] = serie.get("nombre", pd.NA)
                data[f"Resultado_{s_idx}"] = serie.get("resultado", pd.NA)
                variables = serie.get("variables", [])
                if not isinstance(variables, list):
                    variables = []
                for v_idx, var in enumerate(variables, start=1):
                    data[f"Var_Valor_{s_idx}_{v_idx}"] = var.get("valor", pd.NA)
                    data[f"Var_Nombre_{s_idx}_{v_idx}"] = var.get("nombre", pd.NA)
                    data[f"Var_Simbolo_{s_idx}_{v_idx}"] = var.get("simbolo", pd.NA)
            return pd.Series(data)

        expanded_multi = series_list.apply(expand_multiserie_row)
        df_todos.loc[mask_multi, expanded_multi.columns] = expanded_multi.values

# 7) Guardar resultado
df_todos.to_excel("Consolidado_Serie_Multi.xlsx", index=False)
print("✅ Consolidado.xlsx generado con columnas optimizadas para 'Serie Unica' y 'Multiserie'.")

✅ Consolidado.xlsx generado con columnas optimizadas para 'Serie Unica' y 'Multiserie'.
